In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import re
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.fundamentaldata import FundamentalData
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data as wb

In [ ]:
#API Key
key = "6Y9HBNUKXJKOL6D4"

#Get Data
ts = TimeSeries(key, output_format = "pandas")
ti = TechIndicators(key, output_format = "pandas")
fd = FundamentalData(key, output_format = "pandas")

In [ ]:
#Calculating a securities risk vs. Market
tickers = ["MSFT", "^GSPC"]

comp_data = pd.DataFrame()

for t in tickers:
    comp_data[t] = wb.DataReader(t, data_source = "yahoo", start = "2011-1-1")["Adj Close"]

In [ ]:
(comp_data / comp_data.iloc[0] * 100).plot(figsize(15, 6));

plt.show()

In [ ]:
#Calculating a securities risk 
tickers = ["MSFT", "AAPL"]

comp_data_2 = pd.DataFrame()

for t in tickers:
    comp_data_2[t] = wb.DataReader(t, data_source = "yahoo", start = "2011-1-1")["Adj Close"]

In [ ]:
#comp_data.tail()

In [ ]:
comp_returns = np.log(comp_data_2 / comp_data_2.shift(1))

In [ ]:
#comp_returns

In [ ]:
#Daily average return over 250 trading days
comp_returns[["MSFT", "AAPL"]].mean() * 250

In [ ]:
#Standard Deviation
comp_returns[["MSFT", "AAPL"]].std() * 250 ** 0.5

In [ ]:
#Ticker symbol
x = "MSFT"

In [ ]:
daily_adjusted_data, meta_data = ts.get_daily_adjusted(symbol="x")#, outputsize = "full")

In [ ]:
daily_adjusted_data = pd.DataFrame(daily_adjusted_data)

In [ ]:
adjusted_diff = daily_adjusted_data["5. adjusted close"].diff()

In [ ]:
reverse_daily = daily_adjusted_data.iloc[::-1]

In [ ]:
rev_adjusted_diff = reverse_daily["5. adjusted close"].diff()

In [ ]:
rev_adjusted_df = rev_adjusted_diff.to_frame()

In [ ]:
daily_adjusted_df = pd.merge(daily_adjusted_data, rev_adjusted_df, how = "inner", on = "date")

In [ ]:
daily_adjusted_df = daily_adjusted_df.rename(columns={"1. open":"Open", "2. high":"High", "3. low":"Low", "4. close":"Close", "5. adjusted close_x":"Adjusted_Close", "6. volume":"Volume", "7. dividend amount":"Dividend_Amount", "8. split coefficient":"Split_Coefficient", "5. adjusted close_y":"Difference_Adjusted_Close_(%)"})

In [ ]:
daily_adjusted_df.reset_index(inplace=True)

In [ ]:
#Overview of company
#Company_overview, meta_data = fd.get_company_overview(symbol="x")

In [ ]:
#DAILY ADJUSTED 

fig = go.Figure(data=[go.Candlestick(x=daily_adjusted_df["date"],
               open=daily_adjusted_df["Open"],
               high=daily_adjusted_df["High"],
               low=daily_adjusted_df["Low"],
               close=daily_adjusted_df["Close"])])




fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.update_traces(increasing_line_color="green", selector=dict(type='candlestick'))
fig.show()